In [22]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator
from qiskit.quantum_info.operators import Operator
from qiskit.visualization import plot_histogram

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2022-10-17 01:11:38,367: Credentials are already in use. The existing account in the session will be replaced.


In [23]:
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    # Do multi-controlled-Z gate
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
    qc.h(nqubits-1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s

In [32]:
def function(mis_list: list, num):
    if num in mis_list:
        return 0
    else:
        return 1
    
def missing_number(mis_list: list):
    """
    """
    n = int(np.log2(len(mis_list)+1))
    qc = QuantumCircuit(n)
    for i in range(n):
        qc.h(i)
    qc.barrier()
    matrix = []
    for i in range(2**n):
        row = [0 for i in range(2**n)]
        f_i = function(mis_list, i)
        row[i] = (-1)**f_i
        matrix.append(row)
    
    oracle = Operator(matrix)
    qc.unitary(oracle, [qubit for qubit in range(n)], label='oracle')
    
    qc.barrier()
    qc.append(diffuser(n), [qubit for qubit in range(n)])
    qc.measure_all()
    qc.draw()
    print(qc)
    
    aer_sim = Aer.get_backend('aer_simulator')
    transpiled_qc = transpile(qc, aer_sim)
    qobj = assemble(transpiled_qc)
    results = aer_sim.run(qobj).result()
    counts = results.get_counts()
    plot_histogram(counts)

In [33]:
missing_number([3,1,0])

        ┌───┐ ░ ┌─────────┐ ░ ┌────────┐ ░ ┌─┐   
   q_0: ┤ H ├─░─┤0        ├─░─┤0       ├─░─┤M├───
        ├───┤ ░ │  oracle │ ░ │  U$_s$ │ ░ └╥┘┌─┐
   q_1: ┤ H ├─░─┤1        ├─░─┤1       ├─░──╫─┤M├
        └───┘ ░ └─────────┘ ░ └────────┘ ░  ║ └╥┘
meas: 2/════════════════════════════════════╩══╩═
                                            0  1 
Traceback (most recent call last):
  Input In [33] in <cell line: 1>
    missing_number([3,1,0])
  Input In [32] in missing_number
    qobj = assemble(transpiled_qc)
NameError: name 'assemble' is not defined

Use %tb to get the full traceback.
